<a href="https://colab.research.google.com/github/atlas-github/fi_analytics/blob/master/Chapter_5_Sentiment_analysis_using_Lowyat_NET_comments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Scrape

Now that the sections of the HTML code are known, start by scraping the first page of the thread.

In [1]:
import requests
import urllib.request
from bs4 import BeautifulSoup
import pandas as pd

url = "https://forum.lowyat.net/topic/4828500"

response = requests.get(url)
response

<Response [200]>

Scrape the comments next. Comments are in **`<div class="postcolor post_text" data-postid="93743887">Lod incoming<br> <!--IBF.ATTACHMENT_93743887--></div>`** (i.e. div class called postcolor post_text).

#Parse

Use `BeautifulSoup`’s `html.parser` to look for all `div` classes named `postcolor post_text`. Get the text within the HTML data using `get_text(). comments_list` should resemble the results as follows. 

In [2]:
soup = BeautifulSoup(response.text, "html.parser")
comments = soup.findAll("div",{'class':"postcolor post_text"})

comments_temp = list(comments)

comments_list = []

for i in range(len(comments)):
    comments_list.append(comments[i].get_text())

comments_list

['    Bloody useless Octopus Bank.Early they said the apps is down and ask us to use Web version, now the web version is down.  I want to transfer money right now also cannot.This post has been edited by cringe: Aug 26 2019, 10:39 PM ',
 'Lod incoming ',
 'could it be another security issue? jeng jeng jeng.gif ',
 "QUOTE(Lyu @ Aug 26 2019, 10:38 PM)Lod incomingWhat LOD. I'm Getting frustrated with this bank. ",
 'Yup. Progressively getting worse with a major acceleration this year. ',
 "QUOTE(cringe @ Aug 26 2019, 10:42 PM)What LOD. I'm Getting frustrated with this bank.Y limited thee self to one...? ",
 'QUOTE(Lyu @ Aug 26 2019, 10:38 PM)Lod incomingLyu, you applied CC already? ',
 "QUOTE(Baconateer @ Aug 26 2019, 10:42 PM)could it be another security issue? jeng jeng jeng.gifUsing KLIA's system. Plot twist. ",
 'Its been down since morning. Only got to use the web version at abt 7+. Trf my money out to rhb in case app down again. ',
 'Ive said it again n againMb2u > all ',
 'one is t

Now go for the usernames. Usernames are in **`<span class="normalname"><a href="/user/Lyu">Lyu</a></span>`** (i.e. span class called normalname). Modify the `soup.findAll` code to look for all `span` classes named `normalname`.

In [6]:
soup2 = BeautifulSoup(response.text, "html.parser")
names = soup2.findAll("span", {"class": "normalname"})
names_temp = list(names)
names_list = []

for i in range(len(names)):
    names_list.append(names[i].get_text())

names_list

['TScringe',
 'Lyu',
 'Baconateer',
 'TScringe',
 'thesoothsayer',
 'Lyu',
 'JoeK',
 'sp6068',
 'CWW88',
 'ketupatlazat',
 'alexkos',
 'TScringe',
 'alexkos',
 'KcX35',
 'Lyu',
 'judehow',
 'robert82',
 'JoeK',
 'TScringe',
 'robert82']

Now that the usernames and comments have been obtained, merge both lists into a Python dataframe. 

In [7]:
compiled_list = list(zip(names_list, comments_list))
dataframe = pd.DataFrame(compiled_list, columns = ["Person", "Comment"])
dataframe

,Person,Comment
0,TScringe,Bloody useless Octopus Bank.Early they sai...
1,Lyu,Lod incoming
2,Baconateer,could it be another security issue? jeng jeng ...
3,TScringe,"QUOTE(Lyu @ Aug 26 2019, 10:38 PM)Lod incoming..."
4,thesoothsayer,Yup. Progressively getting worse with a major ...
5,Lyu,"QUOTE(cringe @ Aug 26 2019, 10:42 PM)What LOD...."
6,JoeK,"QUOTE(Lyu @ Aug 26 2019, 10:38 PM)Lod incoming..."
7,sp6068,"QUOTE(Baconateer @ Aug 26 2019, 10:42 PM)could..."
8,CWW88,Its been down since morning. Only got to use t...
9,ketupatlazat,Ive said it again n againMb2u > all


Your table should look like the snapshot above. Now that we have the scraping algorithm, the `url` needs to be modified, since this thread has multiple pages. 

For page 1, the URL is https://forum.lowyat.net/topic/4828500

For page 2, the URL is https://forum.lowyat.net/topic/4828500/+20

For page 3, the URL is https://forum.lowyat.net/topic/4828500/+40

There’s a pattern in the URL. Notice the end of the URL increases by 20 for subsequent pages. This must be some sort of comment count, since every page contains 20 comments. 

The URL modification can be achieved as follows. 


In [8]:
import requests
import urllib.request
from bs4 import BeautifulSoup
import pandas as pd

pages = ["/+0", "/+20", "/+40", "/+60"]
comments_list = []
names_list = []

for i in range(0, len(pages)):
  url = "https://forum.lowyat.net/topic/4828500"+pages[i]
  response = requests.get(url)
  soup = BeautifulSoup(response.text, "html.parser")
  comments = soup.findAll("div",{'class':"postcolor post_text"})
  comments_temp = list(comments)

  for i in range(len(comments_temp)):
    comments_list.append(comments_temp[i].get_text())

  names = soup.findAll("span", {"class": "normalname"})
  names_temp = list(names)

  for i in range(len(names_temp)):
    names_list.append(names_temp[i].get_text())

compiled_list = list(zip(names_list, comments_list))

dataframe = pd.DataFrame(compiled_list, columns = ["Person", "Comment"])

dataframe

,Person,Comment
0,TScringe,Bloody useless Octopus Bank.Early they sai...
1,Lyu,Lod incoming
2,Baconateer,could it be another security issue? jeng jeng ...
3,TScringe,"QUOTE(Lyu @ Aug 26 2019, 10:38 PM)Lod incoming..."
4,thesoothsayer,Yup. Progressively getting worse with a major ...
...,...,...
58,lrtss18,say no to cimbeng
59,POYOZER,Omg. My bills still pending.
60,andise,still slow until today
61,graphidz,Remember not to keep all your money in one ban...


Notice again, the code built previously served as a “foundation” for the code above. This feature is common across many data projects. Whenever you are unsure whether you can achieve a certain task, break it down by trying to get a small portion of the data first. Once this is completed, scale up your code to get all the data. 

Notice this dataframe has 63 rows. It indicates you now have all the comments from this particular thread. 

#Sentiment Analysis

Now for the exciting part. The comments have been obtained - they need to be classified (into Positive, Negative, or Neutral) and scored (i.e. not all negative comments are the same as some will be more serious than others, and likewise for positive comments). 

There are multiple libraries available for sentiment analysis of English comments: 

1.   [NLTK’s sentiment](https://www.nltk.org/howto/)
2.   [Google Cloud Natural Language API’s Sentiment Analysis](https://cloud.google.com/natural-language/docs/sentiment-tutorial)
3.   [scikit-learn’s classifiers](https://scikit-learn.org/stable/tutorial/text_analytics/working_with_text_data.html)

and so on.

For this simple demonstration, I’ll be using the [afinn](https://github.com/fnielsen/afinn) classifier to obtain some business insights. Once this has been achieved, do feel free to test out other Python libraries for improved results. 

In your IDE, it’s likely afinn is not installed yet. In Google Colab, run the following code to install and initialize afinn.

In [9]:
!pip install afinn
from afinn import Afinn
af = Afinn()

     |████████████████████████████████| 61kB 1.8MB/s 
  Created wheel for afinn: filename=afinn-0.1-cp36-none-any.whl size=53452 sha256=94bcda66e9633b4d6ff09f1d05bd5d5f83b9c1a996a46b87d317b0e748a81cdc
  Stored in directory: /root/.cache/pip/wheels/b5/1c/de/428301f3333ca509dcf20ff358690eb23a1388fbcbbde008b2
Successfully built afinn


The next step is to create an empty array to store the sentiment results. Do create a sentiment category too. It’s to classify as `positive` comments which have scores above 0, `negative` for scores below 0, and `neutral` for scores which are 0. 

In [10]:
sentiment_scores = []

for i in range(len(dataframe)):
  sentiment_scores.append(af.score(dataframe.iloc[i, 1]))

sentiment_category = ['positive' if score > 0 
                           else 'negative' if score < 0 
                               else 'neutral' 
                                   for score in sentiment_scores]

sentiment_scores

[-4.0,
 0.0,
 0.0,
 -2.0,
 -3.0,
 -3.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 -1.0,
 -1.0,
 1.0,
 2.0,
 -4.0,
 -2.0,
 3.0,
 0.0,
 0.0,
 3.0,
 -2.0,
 -2.0,
 0.0,
 6.0,
 0.0,
 0.0,
 0.0,
 -3.0,
 3.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 -3.0,
 0.0,
 -3.0,
 1.0,
 0.0,
 0.0,
 3.0,
 0.0,
 1.0,
 1.0,
 0.0,
 -2.0,
 2.0,
 -2.0,
 0.0,
 -2.0,
 1.0,
 2.0,
 10.0,
 0.0,
 -2.0,
 0.0,
 1.0,
 -1.0,
 0.0,
 0.0,
 -2.0,
 0.0]

The next step is to merge `sentiment_scores` and `sentiment_category` as new columns to `dataframe`. The steps to do that are as follows:

1.   Turn `sentiment_scores` into a dataframe
2.   Turn `sentiment_category` into a dataframe
3.   Merge `sentiment_scores` and `sentiment_category` to dataframe
4.   Display the merged dataframe and export the file as csv

In [12]:
sentiment_df = pd.DataFrame(sentiment_scores, columns = {'Sentiment_Scores'})

category_df = pd.DataFrame(sentiment_category, columns = {"Sentiment Category"})

df = pd.concat([dataframe, sentiment_df, category_df], axis = 1)

df['Sentiment_Scores'] = df.Sentiment_Scores.astype('float')

df

,Person,Comment,Sentiment_Scores,Sentiment Category
0,TScringe,Bloody useless Octopus Bank.Early they sai...,-4.0,negative
1,Lyu,Lod incoming,0.0,neutral
2,Baconateer,could it be another security issue? jeng jeng ...,0.0,neutral
3,TScringe,"QUOTE(Lyu @ Aug 26 2019, 10:38 PM)Lod incoming...",-2.0,negative
4,thesoothsayer,Yup. Progressively getting worse with a major ...,-3.0,negative
...,...,...,...,...
58,lrtss18,say no to cimbeng,-1.0,negative
59,POYOZER,Omg. My bills still pending.,0.0,neutral
60,andise,still slow until today,0.0,neutral
61,graphidz,Remember not to keep all your money in one ban...,-2.0,negative


In [ ]:
df.to_csv("complaints.csv")
from google.colab import files
files.download("complaints.csv")